In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
from queue import Queue
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
import requests

In [2]:
# Função para pegar os links do botão detalhar
def get_href(elements):
    hrefs = []
    for elemento in elements:
        href = elemento.get_attribute('href')
        hrefs.append(href)

    return hrefs

# Função para acessar o portal com os parametros desejados
def consultar(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    municipio = quote(municipio)
    navegador.get(f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc')

# Função para verificar se o número da página é maior que o valor anteriormente armazenado
def page_number_greater(driver, previous_page, id = 'lista_info'):
    page_text = driver.find_element(By.ID, id).text
    current_page = int(page_text.split()[1])
    return current_page > previous_page

In [ ]:
beneficio = 'bolsa-familia'
start_date = '01/01/2020'
end_date = '31/12/2020'
municipio = 'Tanguá'

# Iniciando drive do Chorme
driver = webdriver.Chrome()

# inicia a pagiana e espera 5 segundos para os dados serem carregados
consultar(driver, beneficio, start_date, end_date, municipio)
sleep(5)

# Clica no botão de ver paginação completa
driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()
sleep(5)

# Seleciona a opção de 50 individuos por páginas
elemento_dropdown  = driver.find_element(By.NAME, 'lista_length')
Select(elemento_dropdown).select_by_value('50')
sleep(5)

# Verifica quantas paginas no total
quantPaginas = int(driver.find_element(By.ID, 'lista_info').text.split()[-1])

# Crie uma lista para armazenar os valores de atributo href
hrefs = []
previous_page = 1
for _ in range(quantPaginas):
    # Busca os lnkes do detalhar
    elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')

    # Tenta obeter o link, caso a requisição demore para responder, espere mais 5 segundos
    try:
        link = get_href(elements)
    except:
        sleep(5)
        elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')
        link = get_href(elements)


    # Adiciona na lista de links
    hrefs.extend(link)
    if previous_page != quantPaginas:
        # Clica no botão proximo  
        driver.find_element(By.ID, 'lista_next').click()

        # Aguardar a mudança de página
        WebDriverWait(driver, 10).until(lambda driver: page_number_greater(driver, previous_page))
        previous_page += 1

driver.quit()

## Transformando links em csv

In [39]:
df = pd.DataFrame(hrefs, columns=['Detalhes_link'])

In [40]:
ano = start_date.split('/')[-1]

df.to_csv(f'{start_date}.csv')

In [105]:
df = pd.read_csv('2017.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv('bolsa-familia-2017.csv')

In [176]:
# Lista para armazenar os dados dos arquivos CSV
dados_csv = []

# Percorra todos os arquivos na pasta
for nome_arquivo in os.listdir('Dados/'):
    if nome_arquivo.endswith('.csv'):
        # Construa o caminho completo para cada arquivo CSV
        caminho_arquivo = os.path.join('Dados/', nome_arquivo)
        
        # Leia o arquivo CSV e adicione os dados à lista
        df = pd.read_csv(caminho_arquivo)
        dados_csv.append(df)

# Concatene todos os DataFrames em um único DataFrame
dados_completos = pd.concat(dados_csv)

In [177]:
dados_completos.drop('Unnamed: 0', axis=1, inplace=True)
dados_completos.drop_duplicates(inplace=True)
dados_completos.reset_index(drop=True, inplace=True)
dados_completos.to_csv('Dados/bolsa-familia.csv', index=False)

# Teste de extrair quantidade da tabela

In [20]:
driver = webdriver.Chrome()
driver.get('https://portaldatransparencia.gov.br/beneficios/bolsa-familia/301738618?ordenarPor=mesReferencia&direcao=desc')
sleep(3)
user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]
name = user_data.get_attribute("innerHTML").split()[-1]


In [37]:
df = pd.read_csv('Dados/bolsa-familia.csv')
link_list = df.values.tolist()

In [72]:
driver = webdriver.Chrome()
driver.maximize_window()
error_link = []
dados = []

# Percorre cada link
for link in link_list:
    driver.get(link[0])
    sleep(3)

    # Localize o NIS
    try:
        user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]
    except:
        error_link.append(link[0])  # Pegua o link que deu erro
        driver.quit()               # Finaliza a instancia do chorme
        sleep(60)                   # Aguarda caso seja bloqueio do sistema
        driver = webdriver.Chrome() # Cria uma nova instancia
        driver.maximize_window()    # Maximiza a tela
        driver.get(link[0])         # Acessa o link
        sleep(3)                    # Aguarda pagina carregar
        user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]

    nis = user_data.get_attribute("innerHTML").split()[-1]

    # Localize a tabela geral
    tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_wrapper')
    tabela.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()
    sleep(5)

    # Seleciona a opção de 50 individuos por páginas
    elemento_dropdown  = driver.find_element(By.NAME, 'tabelaDetalheValoresRecebidos_length')
    Select(elemento_dropdown).select_by_value('50')
    sleep(5)

    # Verifica quantas paginas no total
    quantPaginas = int(driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_info').text.split()[-1])
    
    # Pega a tabela principal
    tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos')
    
    previous_page = 1
    for _ in range(quantPaginas):

        # Gambiarra para tirar o mouse de tootip em cima dos dados
        driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_info').click()
        sleep(0.1)

        # Obtenha todas as linhas da tabela
        linhas = tabela.find_elements(By.TAG_NAME, 'tr')[1:]

        # Itere sobre as linhas
        for linha in linhas:
            # Obtenha todas as células da linha
            celulas = linha.find_elements(By.TAG_NAME, 'td')
            
            # Extraia os dados das células
            dados_linha = [celula.text for celula in celulas]

            # Adicionando o nis
            dados_linha.append(nis)

            # Armazene os dados
            dados.append(dados_linha)

        if previous_page != quantPaginas:
            # Clica no botão proximo  
            driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_next').click()

            # Aguardar a mudança de página
            WebDriverWait(driver, 10).until(lambda driver: page_number_greater(driver, previous_page, 'tabelaDetalheValoresRecebidos_info'))
            previous_page += 1
        
# Feche o navegador
driver.quit()

peguei


KeyboardInterrupt: 

In [68]:
# Começar partir daqui
linha = df[df['Detalhes_link'] == 'https://portaldatransparencia.gov.br/beneficios/bolsa-familia/299403958'].index[0]

In [75]:
linha

697

In [ ]:
link_list[linha:]

In [50]:
header = ['Mes Folha', 'Mes Referência', 'UF', 'Municipio', 'Quantidade Dependentes', 'Valor', 'NIS']
pd.DataFrame(dados, columns=header).to_csv('Dados/detalhes-bolsa-familia.csv')

# Usando API

In [27]:
def obter_dados_auxilio_emergencial(codigo_ibge, mes_ano, beneficio):
    url = f"https://api.portaldatransparencia.gov.br/api-de-dados/{beneficio}-disponivel-beneficiario-por-municipio"
    headers = {"chave-api-dados": "07e464f446e88bb0c105f9cea69b7f45"}

    page = 1
    dados = []

    while True:
        params = {
            "codigoIbge": codigo_ibge,
            "mesAno": mes_ano,
            "pagina": str(page)
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        data = response.json()
        if not data:
            break

        for item in data:
            dados.append([
                item['dataMesReferencia'],
                item['municipio']['nomeIBGE'],
                item['titularBolsaFamilia']['cpfFormatado'],
                item['titularBolsaFamilia']['nis'],
                item['titularBolsaFamilia']['nome'],
                item['quantidadeDependentes'],
                item['valor']
            ])

        page += 1
    return dados

In [ ]:
header = ['Mes Referência', 'Município', 'CPF Beneficiário', 'NIS Beneficiário', 'Beneficiário', 'Quantidade Dependentes', 'Valor']

# Utilização da função para obter os dados para todos os meses do ano
try:
    codigo_ibge = "3305752"
    beneficio = 'bolsa-familia'
    anos = [str(ano) for ano in range(2016,2024)] # Adicione todos os anos que deseja incluir
    meses = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]  # Todos os meses do ano

    for ano in anos:
        for mes in meses:
            mes_ano = ano + mes
            dados = obter_dados_auxilio_emergencial(codigo_ibge, mes_ano, beneficio)
            
            # Se os dados não estiverem vazios
            if dados:
                # Cria a pasta para o ano, caso ela não exista
                pasta_ano = f"Dados/{beneficio}/{ano}"
                if not os.path.exists(pasta_ano):
                    os.makedirs(pasta_ano)

                # Salva os dados em um arquivo CSV
                nome_arquivo = f"{mes_ano}_{beneficio}.csv"
                caminho_arquivo = os.path.join(pasta_ano, nome_arquivo)
                df = pd.DataFrame(dados, columns=header)
                df.to_csv(caminho_arquivo, index=False)
                print(f"Dados para o mês {mes} do ano {ano} foram salvos.")
            else:
                print(f"Dados para o mês {mes} do ano {ano} estava vazio.")
            

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err}")
except Exception as err:
    print(f"Erro: {err}")